<h1>Hip roof</h1>

<h2>Simple reference model</h2>
![alt text](reference_model_simplex.gif)

<h2>Complex reference model</h2>
![alt text](reference_model_complex.jpg)

<h4>Variables:</h4>  <b>skel</b>: input hpc's 1 skeleton, 
                     <b>skeleton</b>: skeleton's offset, 
                     <b>skeleton2</b>: ukpol of 2 skeleton of hpc, 
                     <b>vert</b>: verts's list of hpc's 2-skeleton, 
                     <b>cell</b>: cells's list of hpc's 2-skeleton, 
                     <b>final</b>: list to sostitute the verts's index with their real values, 
                     <b>vertex</b>: support's list to new verts's list updated, 
                     <b>cells</b>: updated cells's list, 
                     <b>vertex2</b>: updated verts's list, 
                     <b>coverMkp</b>: cover makepol by verts and cells, 
                     <b>coverStr</b>: cover traslate on z, 
                     <b>coverOff</b>: offset cover, 
                     <b>roof</b>: struct of skeleton and cover to create the final roof in output.

<h4>Description of the geometric methods i'm going to implement:</h4>
Before the definition of the main function, I used a support function to create the skeleton of the roof, which takes as input a hpc created as MKPOL of vertices and early defined cells. I memorize the 1-skeleton of the hpc (roof structure), the offset 0.2x0.2x0.4 of the 1-skeleton, and UKPOL of the 2-skeleton of hpc (values ​​of vertices and convex cells) , then divide the lists of vertices and cells. With UKPOL I have noticed that there are many values ​​than those profits, many of which are repeated values ​​or useless, so the goal of this function is to clean up these lists as the values ​​are not needed, so I use several support lists . Before I used a list of cells where the values ​​of the indices are replaced with the values ​​that really assume, that xyz (final), then this list is cleaned by duplication (vertex), then the list of the cells was clean by not precise values or useless for our purposes, it was finally clean the list of vertices of those points that are the projection of the vertices that have a z> 0. This function returns a list consisting of clean verts, clean cells and the skeleton.
Finally, the main function takes as input a list of vertices and a list of cells defined at the beginning, with these values ​​through MKPOL creating the hpc object that is given as input to the previous function thus allowing the cleaning of the vertices and cells, finally on this clean structure will be given the coverage that will form the final roof.

Import library and define simple and complex verts and cells for the definition of the hip roof

In [9]:
from pyplasm import *

verts_simple = [[0,0,0],[20,0,0],[20,10,0],[0,10,0],[5,5,3],[15,5,3],[5,5,0],[15,5,0]]
cells_simple = [[1,4,5,7],[1,2,5,6,7,8],[2,6,3,8],[3,4,5,6,7,8]]

verts_complex = [[0,10,0],[12,10,0],[12,0,0],[20,0,0],[16,4,0],[16,15,0],[20,20,0],[0,20,0],[5,15,0],[16,4,3],[16,15,3],[5,15,3]]
cells_complex = [[1,8,9,12],[1,2,6,9,11,12],[2,3,5,6,10,11],[6,7,8,9,11,12],[4,5,6,7,10,11],[3,4,5,10]]

Define support function to clean the lists of verts and cells from the unutil values

In [10]:
"""Input: hpc object
   Output: lists of clean verts, cells and skeleton of the hip roof"""

def clean_lists(hpc):
    
    #generate 1-skeleton and ukpol 2-skeleton of the input hpc, verts and cell of 2-skeleton, and define lists's support
    skel = SKEL_1(hpc)
    skeleton = OFFSET([.2,.2,.4])(skel)      
    skeleton2 = UKPOL(SKEL_2(hpc))    
    vert=skeleton2[0]
    cell=skeleton2[1]
    final=[]
    vertex=[]
    cells=[]
    vertex2=[]
    
    #sostitute  the verts's index with their real values
    for c in cell:
        f=[]
        for v in c:
            f.append(vert[v-1])
        final.append(f)
        
    #delete double values
    for u in final:
        for v in u:
            if v not in vertex:
                vertex.append(v)
            else:
                v=vertex.index(v)
                
    #correct values
    for u in final:
        x=[]
        for v in u:
            x.append(vertex.index(v)+1)
        cells.append(x)
    
    #delete verts that are projection on x-y
    cont=0
    for v in vertex:
        if v[2]>0:
            vertex2.append(v)
        else:
            for v2 in vertex:
                if v2[2]>0:
                    if v2[0]==v[0] and v2[1]==v[1]:
                        vertex2.append(v2)
                        cont=1
            if not(cont):
                vertex2.append(v)
            cont=0
    
    #return clean lists of verts, cells and skeleton
    return [vertex2,cells,skeleton]

Define principle function tha give in input lists of verts and cells, and create the final hip roof in output

In [11]:
"""Input: lists of verts and cells
   Output: hip roof as a STRUCT of cover and skeleton"""

def ggpl_hip_roof(verts, cells):
    
    #generate hpc object starting input, and give it to the precedent function
    hpc = MKPOL([verts,cells,None])
    lists = clean_lists(hpc)
    
    #divide the lists returned by 'clean_lists(hpc)'
    vertex2 = lists[0]
    cell = lists[1]
    skeleton = lists[2]
    
    #generate cover, skeleton and struct all
    coverMkp=MKPOL([vertex2,cell,None])
    coverStr=STRUCT([T(3)(.2),coverMkp])
    coverOff=OFFSET([.2,.2,.4])(coverStr)
    skeleton=COLOR(BLACK)(skeleton)
    roof= STRUCT([coverOff,skeleton]) 
    
    return roof

Launch function

In [12]:
roof = ggpl_hip_roof(verts_complex,cells_complex)
VIEW(roof)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004D02450> >

<h2>Final result (simple)</h2>
![alt text](hip_roof_simple.jpg)
![alt text](hip_roof_simple_under.jpg)

<h2>Final result (complex)</h2>
![alt text](skeleton_complex.jpg)
![alt text](hip_roof_complex.jpg)